## Why_finetuning_was_not_possible_in_LSTM.ipynb

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb

In [ ]:
max_len = 200
vocab_size = 10000
embedding_dim = 128
latent_dim = 256

In [ ]:
(x_train, y_train), _ = tf.keras.datasets.imdb.load_data(num_words=vocab_size)

len(x_train)
len(y_train)

In [ ]:
x_train = x_train[:3000]
y_train = y_train[:3000]

len(x_train)
len(y_train)

len(x_train[0])

In [ ]:
x_train = pad_sequences(x_train, maxlen=max_len, padding='post', truncating='post')

In [ ]:
input_layer = Input(shape=(max_len,))
embedding_layer = Embedding(vocab_size, embedding_dim)(input_layer)
lstm_layer, state_h, state_c = LSTM(latent_dim, return_state=True)(embedding_layer)
output_layer = Dense(1, activation='sigmoid')(state_h)
classification_model = Model(input_layer, output_layer)

In [ ]:
classification_model.layers
classification_model.summary()

In [ ]:
classification_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classification_model.fit(x_train, y_train, batch_size=64, epochs=1, validation_split=0.1)

In [ ]:
sample_review = x_train[0].reshape(1, -1)
prediction = classification_model.predict(sample_review)

print("Predicted sentiment probability (positive):", prediction[0][0])
print("Predicted Sentiment: ", "Positive" if prediction[0][0] > 0.5 else "Negative")

In [ ]:
word_index = imdb.get_word_index()
reverse_word_index = {index + 3: word for word, index in word_index.items()}

reverse_word_index[0] = ""
reverse_word_index[1] = ""
reverse_word_index[2] = ""
reverse_word_index[3] = ""

In [ ]:
decoded_review = "".join([reverse_word_index.get(i, "") for i in sample_review[0]])
print("****Decoded Review:****")
print(decoded_review)

In [ ]:
classification_model.save("lstm_imdb_model.h5")

In [ ]:
max_len = 200
vocab_size = 1000
embedding_dim = 128
latent_dim = 256

In [ ]:
(x_train, y_train), _ = tf.keras.datasets.imdb.load_data(num_words=vocab_size)
x_train = pad_sequences(x_train, maxlen=max_len, padding='post', truncating='post')

In [ ]:
x_train = x_train[:1000]
y_train = y_train[:1000]

In [ ]:
load_classification_model = load_model("lstm_imdb_model.h5")
load_classification_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
load_classification_model.fit(x_train, y_train, batch_size=64, epochs=1, validation_split=0.1)
load_classification_model.save("lstm_imdb_model_updated.h5")

In [ ]:
updated_classification_model = load_model("lstm_imdb_model_updated.h5")

encoder_inputs = Input(shape=(max_len,))
encoder_embedding = updated_classification_model.layers[1](encoder_inputs)
encoder_outputs, state_h, state_c = updated_classification_model.layers[2](encoder_embedding)

In [ ]:
output_vocab_size = 8000
target_max_len = 50
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(output_vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=[state_h, state_c])
decoder_dense = Dense(output_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
seq2seq_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
seq2seq_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
encoder_input_data = x_train[:1000]
decoder_input_data = np.random.randint(1, output_vocab_size, (1000, target_max_len))
decoder_target_data = np.random.randint(1, output_vocab_size, (1000, target_max_len, 1))

In [ ]:
encoder_input_data
decoder_input_data
decoder_target_data

In [ ]:
seq2seq_model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=32,
    epochs=1,
    validation_split=0.2
)